<h1>Based on Model 2</h1>
Included Cabin and Passenger Id and Last Name
<br>
<b>Best Score : 0.80476</b>
<b>Rank : 377</b>
on submission_6.csv

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as TSSplit
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,OrdinalEncoder

In [3]:
X_train = pd.read_csv("Train_X.csv")
train_data = pd.read_csv("train.csv")
X_test = pd.read_csv("Test_X.csv")
test_data = pd.read_csv("test.csv")
cabins = train_data.Cabin

In [6]:
cate = []
freq = {}
cabins = test_data.Cabin
for count , i in enumerate(cabins):
    i = str(i)
    cabin = i[0]
    if cabin in freq:
        freq[cabin] +=1
    else:
        freq[cabin] = 1
    cate.append(cabin)
print(freq)
cabins = pd.Series(cate)
cabins.head()

{'G': 1222, 'F': 1445, 'C': 355, 'B': 362, 'D': 242, 'E': 447, 'n': 100, 'A': 98, 'T': 6}


0    G
1    F
2    C
3    C
4    F
dtype: object

In [5]:
enc = LabelEncoder()
cabins_encoded = enc.fit_transform(cabins)
cabins_encoded_s = pd.Series(cabins_encoded,name = 'Cabin')
cabins_encoded_s.head()

0    1
1    5
2    0
3    0
4    5
Name: Cabin, dtype: int32

In [7]:
test_cabins_encoded = enc.transform(cabins)
test_cabins_encoded[:5]

array([6, 5, 2, 2, 5])

In [8]:
X_train_new = X_train.copy()
X_train_new = pd.concat([X_train_new,cabins_encoded_s],axis = 1)
X_train_new.head()

,Earth,Europa,Mars,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin
0,0.0,1.0,0.0,0.0,0.0,1.0,False,39.0,False,0.0,0.0,0.0,0.0,0.0,1
1,1.0,0.0,0.0,0.0,0.0,1.0,False,24.0,False,109.0,9.0,25.0,549.0,44.0,5
2,0.0,1.0,0.0,0.0,0.0,1.0,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,0
3,0.0,1.0,0.0,0.0,0.0,1.0,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,0
4,1.0,0.0,0.0,0.0,0.0,1.0,False,16.0,False,303.0,70.0,151.0,565.0,2.0,5


In [9]:
X_test_new = X_test.copy()
X_test_new = pd.concat([X_test_new,pd.Series(test_cabins_encoded,name = 'Cabin')],axis = 1)
X_test_new.head()

,Earth,Europa,Mars,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin
0,1.0,0.0,0.0,0.0,0.0,1.0,True,27.0,False,0.0,0.0,0.0,0.0,0.0,6
1,1.0,0.0,0.0,0.0,0.0,1.0,False,19.0,False,0.0,9.0,0.0,2823.0,0.0,5
2,0.0,1.0,0.0,1.0,0.0,0.0,True,31.0,False,0.0,0.0,0.0,0.0,0.0,2
3,0.0,1.0,0.0,0.0,0.0,1.0,False,38.0,False,0.0,6652.0,0.0,181.0,585.0,2
4,1.0,0.0,0.0,0.0,0.0,1.0,False,20.0,False,10.0,0.0,635.0,0.0,0.0,5


In [10]:
from xgboost import XGBClassifier
from datetime import datetime
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))
xgb = XGBClassifier(learning_rate=0.02, objective='binary:logistic')
params = {
    'n_estimators' : [100,200,300,400,500,600,700,800,900,1000],
    'max_depth' : [3,4,5,6,7],
    'min_child_weight': [1, 5, 10]
}


In [11]:
y_train = train_data.iloc[:,-1]
y_train.head()

0    False
1     True
2    False
3    False
4     True
Name: Transported, dtype: bool

In [28]:
from sklearn.model_selection import GridSearchCV
start_time = timer(None)
grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='roc_auc', n_jobs=-1, cv=5, verbose=3 )
grid.fit(X_train_new,y_train)
timer(start_time)
# print('\n All results:')
# print(grid.cv_results_)
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best score:')
print(grid.best_score_ * 2 - 1)
print('\n Best parameters:')
print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)

Fitting 5 folds for each of 150 candidates, totalling 750 fits

 Time taken: 0 hours 2 minutes and 2.0 seconds.

 Best estimator:
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=10, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

 Best score:
0.7716466048213917

 Best parameters:
{'max_depth': 5, 'min_child_weight': 10, 'n_estimators': 500}


In [29]:
y_test = grid.best_estimator_.predict(X_test_new)
print(y_test[:10])

[1 0 1 1 1 1 1 1 1 1]


In [31]:
results_df = pd.DataFrame(data={'PassengerId':test_data['PassengerId'], 'Transported':y_test})
results_df.Transported = results_df.Transported.astype('bool')
results_df.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [32]:
results_df.to_csv('./submission_3.csv', index=False)

In [33]:
train_data.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [34]:
train_data.Cabin.head(50)

0      B/0/P
1      F/0/S
2      A/0/S
3      A/0/S
4      F/1/S
5      F/0/P
6      F/2/S
7      G/0/S
8      F/3/S
9      B/1/P
10     B/1/P
11     B/1/P
12     F/1/P
13     G/1/S
14     F/2/P
15       NaN
16     F/3/P
17     F/4/P
18     F/5/P
19     G/0/P
20     F/6/P
21     E/0/S
22     E/0/S
23     E/0/S
24     E/0/S
25     E/0/S
26     E/0/S
27     D/0/P
28     C/2/S
29     F/6/S
30     C/0/P
31     F/8/P
32     G/4/S
33     F/9/P
34     F/9/P
35     F/9/P
36     D/1/S
37     D/1/P
38     F/8/S
39    F/10/S
40     G/1/P
41     G/2/P
42     B/3/P
43     G/3/P
44     G/3/P
45     G/3/P
46    F/10/P
47    F/10/P
48     E/1/S
49     E/2/S
Name: Cabin, dtype: object

In [14]:
cate = []
# cabins = test_data.Cabin
cabins = test_data.Cabin
for count , i in enumerate(cabins):
    i = str(i)
    cabin = i[-1]
    if cabin == 'P':
        cate.append(0)
    elif cabin == 'S':
        cate.append(1)
    else:
        cate.append(-1)
cabins = pd.Series(cate,name = "Cabin_Pos")
cabins.head()

0    1
1    1
2    1
3    1
4    1
Name: Cabin_Pos, dtype: int64

In [13]:
X_train_new = pd.concat([X_train_new,cabins],axis = 1)
X_train_new.head()

,Earth,Europa,Mars,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin,Cabin_Pos
0,0.0,1.0,0.0,0.0,0.0,1.0,False,39.0,False,0.0,0.0,0.0,0.0,0.0,1,0
1,1.0,0.0,0.0,0.0,0.0,1.0,False,24.0,False,109.0,9.0,25.0,549.0,44.0,5,1
2,0.0,1.0,0.0,0.0,0.0,1.0,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,0,1
3,0.0,1.0,0.0,0.0,0.0,1.0,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,0,1
4,1.0,0.0,0.0,0.0,0.0,1.0,False,16.0,False,303.0,70.0,151.0,565.0,2.0,5,1


In [15]:
X_test_new = pd.concat([X_test_new,cabins],axis = 1)
X_test_new.head()

,Earth,Europa,Mars,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin,Cabin_Pos
0,1.0,0.0,0.0,0.0,0.0,1.0,True,27.0,False,0.0,0.0,0.0,0.0,0.0,6,1
1,1.0,0.0,0.0,0.0,0.0,1.0,False,19.0,False,0.0,9.0,0.0,2823.0,0.0,5,1
2,0.0,1.0,0.0,1.0,0.0,0.0,True,31.0,False,0.0,0.0,0.0,0.0,0.0,2,1
3,0.0,1.0,0.0,0.0,0.0,1.0,False,38.0,False,0.0,6652.0,0.0,181.0,585.0,2,1
4,1.0,0.0,0.0,0.0,0.0,1.0,False,20.0,False,10.0,0.0,635.0,0.0,0.0,5,1


In [61]:
params = {
    'n_estimators' : [100,200,300,400,500,600,700,800,900,1000],
    'max_depth' : [3,4,5,6,7],
    'min_child_weight': [1, 5, 10,15],
    'learning_rate' : [0.001,0.005,0.01,0.03,0.05,0.075,0.1]
}

In [17]:
xgb2 = XGBClassifier(objective='binary:logistic')
from sklearn.model_selection import GridSearchCV
start_time = timer(None)
grid = GridSearchCV(estimator=xgb2, param_grid=params, scoring='roc_auc', n_jobs=-1, cv=5, verbose=3 )
grid.fit(X_train_new,y_train)
timer(start_time)
# print('\n All results:')
# print(grid.cv_results_)
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best score:')
print(grid.best_score_ * 2 - 1)
print('\n Best parameters:')
print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)

Fitting 5 folds for each of 1400 candidates, totalling 7000 fits

 Time taken: 0 hours 21 minutes and 7.26 seconds.

 Best estimator:
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

 Best score:
0.7770957126303593

 Best parameters:
{'learning_rate': 0.03, 'max_depth': 7, 'min_child_weight': 1, 

In [50]:
y_test = grid.best_estimator_.predict(X_test_new)
results_df = pd.DataFrame(data={'PassengerId':test_data['PassengerId'], 'Transported':y_test})
results_df.Transported = results_df.Transported.astype('bool')
results_df.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [52]:
results_df.to_csv("submission_4.csv",index = False)

In [18]:
train_data.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [20]:
train_data.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [27]:
names = test_data.Name
names = names.fillna('')
last_names = []
for name in names:
    last_names.append(name.split(' ')[-1])
last_names = pd.Series(last_names,name = 'LastName')
last_names.shape,X_train_new.shape

((4277,), (8693, 17))

In [26]:
X_train_new = X_train_new.assign(LastName = last_names)
X_train_new.head()

,Earth,Europa,Mars,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin,Cabin_Pos,LastName
0,0.0,1.0,0.0,0.0,0.0,1.0,False,39.0,False,0.0,0.0,0.0,0.0,0.0,1,0,Ofracculy
1,1.0,0.0,0.0,0.0,0.0,1.0,False,24.0,False,109.0,9.0,25.0,549.0,44.0,5,1,Vines
2,0.0,1.0,0.0,0.0,0.0,1.0,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,0,1,Susent
3,0.0,1.0,0.0,0.0,0.0,1.0,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,0,1,Susent
4,1.0,0.0,0.0,0.0,0.0,1.0,False,16.0,False,303.0,70.0,151.0,565.0,2.0,5,1,Santantines


In [28]:
X_test_new = X_test_new.assign(LastName = last_names)
X_test_new.head()

,Earth,Europa,Mars,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin,Cabin_Pos,LastName
0,1.0,0.0,0.0,0.0,0.0,1.0,True,27.0,False,0.0,0.0,0.0,0.0,0.0,6,1,Carsoning
1,1.0,0.0,0.0,0.0,0.0,1.0,False,19.0,False,0.0,9.0,0.0,2823.0,0.0,5,1,Peckers
2,0.0,1.0,0.0,1.0,0.0,0.0,True,31.0,False,0.0,0.0,0.0,0.0,0.0,2,1,Unhearfus
3,0.0,1.0,0.0,0.0,0.0,1.0,False,38.0,False,0.0,6652.0,0.0,181.0,585.0,2,1,Caltilter
4,1.0,0.0,0.0,0.0,0.0,1.0,False,20.0,False,10.0,0.0,635.0,0.0,0.0,5,1,Harperez


In [45]:
enc = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
t = X_train_new.copy()
encoded_name = np.array(enc.fit_transform(t[['LastName']]))
encoded_name = encoded_name.reshape(-1)
encoded_name = pd.Series(encoded_name,name = 'LastName')
encoded_name.head()

0    1432.0
1    2110.0
2    1991.0
3    1991.0
4    1779.0
Name: LastName, dtype: float64

In [47]:
t_t = X_test_new.copy()
t_encoded = enc.transform(t_t[['LastName']]).reshape(-1)
t_encoded = pd.Series(t_encoded, name = 'LastName')
t_encoded.head()

0     345.0
1    1506.0
2    2071.0
3     325.0
4     927.0
Name: LastName, dtype: float64

In [50]:
X_train_new.LastName = encoded_name
X_test_new.LastName = t_encoded

In [53]:
Ids = train_data.PassengerId
Ids.fillna(0)
cat = []
freq = {}
for id in Ids:
    id = id.split('_')[-1]
    cat.append(id)
    if id in freq:
        freq[id] += 1
    else:
        freq[id] = 1
X_train_new = X_train_new.assign(Id = cat)
X_train_new.head()

,Earth,Europa,Mars,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin,Cabin_Pos,LastName,Id
0,0.0,1.0,0.0,0.0,0.0,1.0,False,39.0,False,0.0,0.0,0.0,0.0,0.0,1,0,1432.0,01
1,1.0,0.0,0.0,0.0,0.0,1.0,False,24.0,False,109.0,9.0,25.0,549.0,44.0,5,1,2110.0,01
2,0.0,1.0,0.0,0.0,0.0,1.0,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,0,1,1991.0,01
3,0.0,1.0,0.0,0.0,0.0,1.0,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,0,1,1991.0,02
4,1.0,0.0,0.0,0.0,0.0,1.0,False,16.0,False,303.0,70.0,151.0,565.0,2.0,5,1,1779.0,01


In [54]:
Ids = test_data.PassengerId
Ids.fillna(0)
cat = []
freq = {}
for id in Ids:
    id = id.split('_')[-1]
    cat.append(id)
    if id in freq:
        freq[id] += 1
    else:
        freq[id] = 1
X_test_new = X_test_new.assign(Id = cat)
X_test_new.head()

,Earth,Europa,Mars,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin,Cabin_Pos,LastName,Id
0,1.0,0.0,0.0,0.0,0.0,1.0,True,27.0,False,0.0,0.0,0.0,0.0,0.0,6,1,345.0,01
1,1.0,0.0,0.0,0.0,0.0,1.0,False,19.0,False,0.0,9.0,0.0,2823.0,0.0,5,1,1506.0,01
2,0.0,1.0,0.0,1.0,0.0,0.0,True,31.0,False,0.0,0.0,0.0,0.0,0.0,2,1,2071.0,01
3,0.0,1.0,0.0,0.0,0.0,1.0,False,38.0,False,0.0,6652.0,0.0,181.0,585.0,2,1,325.0,01
4,1.0,0.0,0.0,0.0,0.0,1.0,False,20.0,False,10.0,0.0,635.0,0.0,0.0,5,1,927.0,01


In [56]:
total_spent = []
required_cols = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
for index,row in train_data.iterrows():
    sum = 0
    for col in required_cols:
        sum += row[col]
    total_spent.append(sum)
X_train_new = X_train_new.assign(TotalSpend = total_spent)
X_train_new.head()

,Earth,Europa,Mars,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin,Cabin_Pos,LastName,Id,TotalSpend
0,0.0,1.0,0.0,0.0,0.0,1.0,False,39.0,False,0.0,0.0,0.0,0.0,0.0,1,0,1432.0,01,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,False,24.0,False,109.0,9.0,25.0,549.0,44.0,5,1,2110.0,01,736.0
2,0.0,1.0,0.0,0.0,0.0,1.0,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,0,1,1991.0,01,10383.0
3,0.0,1.0,0.0,0.0,0.0,1.0,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,0,1,1991.0,02,5176.0
4,1.0,0.0,0.0,0.0,0.0,1.0,False,16.0,False,303.0,70.0,151.0,565.0,2.0,5,1,1779.0,01,1091.0


In [57]:
total_spent = []
required_cols = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
for index,row in test_data.iterrows():
    sum = 0
    for col in required_cols:
        sum += row[col]
    total_spent.append(sum)
X_test_new = X_test_new.assign(TotalSpend = total_spent)
X_test_new.head()

,Earth,Europa,Mars,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin,Cabin_Pos,LastName,Id,TotalSpend
0,1.0,0.0,0.0,0.0,0.0,1.0,True,27.0,False,0.0,0.0,0.0,0.0,0.0,6,1,345.0,01,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,False,19.0,False,0.0,9.0,0.0,2823.0,0.0,5,1,1506.0,01,2832.0
2,0.0,1.0,0.0,1.0,0.0,0.0,True,31.0,False,0.0,0.0,0.0,0.0,0.0,2,1,2071.0,01,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,False,38.0,False,0.0,6652.0,0.0,181.0,585.0,2,1,325.0,01,7418.0
4,1.0,0.0,0.0,0.0,0.0,1.0,False,20.0,False,10.0,0.0,635.0,0.0,0.0,5,1,927.0,01,645.0


In [70]:
X_train_new.to_csv("Train_X.csv",index = False)

In [71]:
X_test_new.to_csv("Test_X.csv",index = False)

In [66]:
X_test_new.Id = X_test_new.Id.astype('int')
X_test_new.dtypes

Earth            float64
Europa           float64
Mars             float64
55 Cancri e      float64
PSO J318.5-22    float64
TRAPPIST-1e      float64
CryoSleep           bool
Age              float64
VIP                 bool
RoomService      float64
FoodCourt        float64
ShoppingMall     float64
Spa              float64
VRDeck           float64
Cabin              int32
Cabin_Pos          int64
LastName         float64
Id                 int32
TotalSpend       float64
dtype: object

In [72]:
y_train.dtype

dtype('bool')

In [73]:
xgb3 = XGBClassifier()
from sklearn.model_selection import GridSearchCV
start_time = timer(None)
grid = GridSearchCV(estimator=xgb3, param_grid=params, scoring='roc_auc', n_jobs=-1, cv=5, verbose=3 )
grid.fit(X_train_new,y_train)
timer(start_time)
# print('\n All results:')
# print(grid.cv_results_)
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best score:')
print(grid.best_score_ * 2 - 1)
print('\n Best parameters:')
print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)

Fitting 5 folds for each of 1400 candidates, totalling 7000 fits

 Time taken: 0 hours 26 minutes and 4.0 seconds.

 Best estimator:
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=900, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

 Best score:
0.7768528470973461

 Best parameters:
{'learning_rate': 0.01, 'max_depth': 6, 'min_child_weight': 1, '

In [68]:
y_test = grid.best_estimator_.predict(X_test_new)
results_df = pd.DataFrame(data={'PassengerId':test_data['PassengerId'], 'Transported':y_test})
results_df.Transported = results_df.Transported.astype('bool')
results_df.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [69]:
results_df.to_csv("submission_5.csv",index = False)

In [81]:
X_train_new.Cabin_Pos = X_train_new.Cabin_Pos.astype('category')
X_train_new.LastName = X_train_new.LastName.astype('category')
X_train_new.Id = X_train_new.Id.astype('category')
X_train_new.Cabin = X_train_new.Cabin.astype('category')
X_train_new.VIP = X_train_new.VIP.astype('category')
X_train_new.CryoSleep = X_train_new.CryoSleep.astype('category')
X_train_new.dtypes

Earth             float64
Europa            float64
Mars              float64
55 Cancri e       float64
PSO J318.5-22     float64
TRAPPIST-1e       float64
CryoSleep        category
Age               float64
VIP              category
RoomService       float64
FoodCourt         float64
ShoppingMall      float64
Spa               float64
VRDeck            float64
Cabin            category
Cabin_Pos        category
LastName         category
Id               category
TotalSpend        float64
dtype: object

In [80]:
X_test_new.Cabin_Pos = X_test_new.Cabin_Pos.astype('category')
X_test_new.LastName = X_test_new.LastName.astype('category')
X_test_new.Id = X_test_new.Id.astype('category')
X_test_new.Cabin = X_test_new.Cabin.astype('category')
X_test_new.VIP = X_test_new.VIP.astype('category')
X_test_new.CryoSleep = X_test_new.CryoSleep.astype('category')
X_test_new.dtypes

Earth             float64
Europa            float64
Mars              float64
55 Cancri e       float64
PSO J318.5-22     float64
TRAPPIST-1e       float64
CryoSleep        category
Age               float64
VIP              category
RoomService       float64
FoodCourt         float64
ShoppingMall      float64
Spa               float64
VRDeck            float64
Cabin            category
Cabin_Pos        category
LastName         category
Id               category
TotalSpend        float64
dtype: object

In [82]:
xgb4 = XGBClassifier(enable_categorical=True)
start_time = timer(None)
grid = GridSearchCV(estimator=xgb4, param_grid=params, scoring='roc_auc', n_jobs=-1, cv=5, verbose=3 )
grid.fit(X_train_new,y_train)
timer(start_time)
# print('\n All results:')
# print(grid.cv_results_)
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best score:')
print(grid.best_score_ * 2 - 1)
print('\n Best parameters:')
print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)

Fitting 5 folds for each of 1400 candidates, totalling 7000 fits

 Time taken: 1 hours 3 minutes and 2.38 seconds.

 Best estimator:
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

 Best score:
0.7814907644036679

 Best parameters:
{'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 5, 'n_e

In [83]:
y_test = grid.best_estimator_.predict(X_test_new)
results_df = pd.DataFrame(data={'PassengerId':test_data['PassengerId'], 'Transported':y_test})
results_df.Transported = results_df.Transported.astype('bool')
results_df.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [84]:
results_df.to_csv('submission_6.csv',index = False)